In [ ]:
import overturemaestro as om
import duckdb
import geopandas as gpd

In [ ]:
pqf = om.convert_geometry_to_parquet(
    "transportation", "segment", geometry_filter=om.geocode_to_geometry("Wrocław")
)
pqf

In [ ]:
# osm_tag_keys = set()
# found_tag_keys = [
#     row[0]
#     for row in self.connection.sql(
#         f"""
#         SELECT DISTINCT UNNEST(map_keys(tags)) tag_key
#         FROM ({parsed_geometries.sql_query()})
#         """
#     ).fetchall()
# ]

In [ ]:
wide_column_definitions = (
    duckdb.sql(
        f"""
    SELECT DISTINCT
        subtype, class, subclass,
        concat_ws('|', subtype, class, subclass) as column_name
    FROM '{pqf}'
    ORDER BY column_name
    """
    )
    .fetchdf()
    .to_dict(orient="records")
)
wide_column_definitions

In [ ]:
condition_columns = ["subtype", "class", "subclass"]
case_clauses = []

for wide_column_definition in wide_column_definitions:
    column_name = wide_column_definition["column_name"]
    conditions = []
    for condition_column in condition_columns:
        if wide_column_definition[condition_column] is None:
            conditions.append(f"{condition_column} IS NULL")
        else:
            # TODO: escape sql value
            escaped_value = wide_column_definition[condition_column]
            conditions.append(f"{condition_column} = '{escaped_value}'")
    case_clauses.append(
        f'CASE WHEN {" AND ".join(conditions)} THEN 1 ELSE 0 END AS "{column_name}"'
    )
case_clauses

In [ ]:
final_query = f"""
    SELECT
        id,
        {", ".join(case_clauses)},
        geometry
    FROM '{pqf}'
"""
duckdb.sql(final_query)

In [ ]:
gpd.read_parquet(pqf)

In [ ]:
pqf2 = om.convert_geometry_to_parquet(
    "places", "place", geometry_filter=om.geocode_to_geometry("Wrocław")
)
pqf2

In [ ]:
gpd.read_parquet(pqf2)

In [ ]:
categories = [
    category_row[0]
    for category_row in duckdb.sql(
        f"""
        SELECT categories['primary'] category
        FROM '{pqf2}'
        UNION
        SELECT UNNEST(categories['alternate']) category
        FROM '{pqf2}'
        """
    ).fetchall()
    if category_row[0]
]
len(categories)

In [ ]:
duckdb.sql(
    f"""
    SELECT DISTINCT categories['primary']
    FROM '{pqf2}'
    """
)

In [ ]:
def sql_escape(value: str) -> str:
    """Escape value for SQL query."""
    return value.replace("'", "''")

In [ ]:
condition_columns = ["subtype", "class", "subclass"]
case_clauses = []

for category in categories:
    escaped_category = sql_escape(category)
    if "s_clothing_store" in category:
        case_clauses.append(
            f"""
            CASE
            WHEN categories['primary'] = '{escaped_category}' THEN 'primary'
            WHEN list_contains(categories['alternate'], '{escaped_category}') THEN 'alternate'
            END AS "{category}"
            """
        )
    

case_clauses[:10]

In [ ]:
final_query = f"""
    SELECT
        id,
        {", ".join(case_clauses)},
        geometry
    FROM '{pqf2}'
"""
duckdb.sql(final_query)

In [ ]:
# osm_tag_keys = set()
# found_tag_keys = [
#     row[0]
#     for row in self.connection.sql(
#         f"""
#         SELECT DISTINCT UNNEST(map_keys(tags)) tag_key
#         FROM ({parsed_geometries.sql_query()})
#         """
#     ).fetchall()
# ]